## Seminar 1. Torch, Backpropagation, My first Neural Network.

### 1. Torch

In [29]:
import numpy as np
import torch
from torch import nn

Main objects in torch are `torch.Tensors` (for storing the data) and `torch.nn.Parameters` (for storing the parameters). A tensor is an object of an arbitrary-dimension (e.g. 1-dim tensor is a vector, 2-dim tensor is a matrix etc.). They can be created the same way as the well-known (for you) `numpy arrays`:

In [30]:
tensor_1 = torch.Tensor([0, 1, 2])
tensor_2 = torch.Tensor(np.array([0, 1, 2]))
tensor_3 = torch.Tensor(range(3))
assert (tensor_1 == tensor_2).all() and (tensor_1 == tensor_3).all()
tensor_1

tensor([0., 1., 2.])

All the base `numpy` functions exist in `torch`:

In [31]:
torch.arange(3), torch.arange(6).reshape(-1, 2).T, torch.var(torch.Tensor([0, 1, 2]))

(tensor([0, 1, 2]),
 tensor([[0, 2, 4],
         [1, 3, 5]]),
 tensor(1.))

We can create torch.Tensors with a pre-defined data type (to reduce the occupied memory):

In [32]:
# See more: https://pytorch.org/docs/stable/tensor_attributes.html
# Int8
tensor_1 = torch.CharTensor([0, 1, 2])
# Int64
tensor_2 = torch.LongTensor([0, 1, 2])
# Float16
tensor_3 = torch.HalfTensor([0, 1, 2])

Change the data type of the tensor:

In [33]:
tensor_4 = tensor_3.to(torch.int8)
tensor_4.dtype

torch.int8

A nice function to select top-k lowest / largest values (which does not exist in `numpy`!):

In [34]:
tensor_4.topk(2)

torch.return_types.topk(
values=tensor([2, 1], dtype=torch.int8),
indices=tensor([2, 1]))

In [35]:
tensor_4.topk(2, largest=False)

torch.return_types.topk(
values=tensor([0, 1], dtype=torch.int8),
indices=tensor([0, 1]))

## Neural-Network-Building Features

A parameter should be created from the `torch.Tensor` of dtype `float / complex`:

In [36]:
tensor = torch.Tensor(range(3))
print(tensor)
param = nn.Parameter(tensor)
param.dtype

tensor([0., 1., 2.])


torch.float32

Parameters accumulate gradients:

In [39]:
param = nn.Parameter(tensor_3)
pre_loss = (param - 1) ** 2
pre_loss = pre_loss.sum()

In [40]:
param.grad

In [41]:
# Calculate gradients for all the parameters which affected the loss value:
pre_loss.backward()

In [42]:
param.grad

tensor([-2.,  0.,  2.], dtype=torch.float16)

### 2. Backpropagation

Consider a dataset with 3 observations and 1 feature:

In [43]:
np.random.seed(42)
X = np.random.uniform(size=3)
y = -np.cos(X)
X, y

(array([0.37454012, 0.95071431, 0.73199394]),
 array([-0.93067597, -0.58110191, -0.74384322]))

Try a linear regression:

In [44]:
from sklearn.linear_model import LinearRegression

In [45]:
linreg = LinearRegression().fit(X[:, None], y)
linreg.coef_

array([0.59850537])

Calculate MSE:

In [46]:
(X * linreg.coef_ + linreg.intercept_ - y)

array([-0.0074582, -0.0121889,  0.0196471])

Let's implement it via manual gradient descent!

In [47]:
torch.manual_seed(42)
W = nn.Parameter(torch.rand(1))
b = nn.Parameter(torch.zeros(1))

In [48]:
torch.Tensor(X).unsqueeze(1)

tensor([[0.3745],
        [0.9507],
        [0.7320]])

In [49]:
X_tensor = torch.Tensor(X).unsqueeze(1)
y_tensor = torch.Tensor(y)

In [50]:
a = X_tensor @ W + b # forward pass result
errors = (a - y_tensor) ** 2 # squared errors
errors

tensor([1.5904, 2.0161, 1.9312], grad_fn=<PowBackward0>)

Calculate the backward pass:
$$\frac{\delta loss}{\delta a} = \frac{\delta ((a - y)^T (a - y) / n)}{\delta a} = \frac{1}{n}(\frac{\delta (a - y)^T}{\delta a}\cdot (a - y) + (a - y)^T \cdot \frac{\delta (a - y)}{\delta a}) = \frac{1}{n} (\frac{\delta (a - y)^T}{\delta a}\cdot (a - y))^T + (a - y)^T = \frac{1}{n} \cdot 2(a - y)^T = \frac{2}{n} (a - y)^T$$

$$\frac{\delta loss}{\delta W} = \frac{\delta loss}{\delta a}\cdot \frac{\delta a}{\delta W}; \frac{\delta a}{\delta W} = \frac{\delta XW + b}{\delta W} = X \Rightarrow \frac{\delta loss}{\delta W} = \frac{2}{n} (a - y)^T \cdot X$$

$$\frac{\delta loss}{\delta b} = \frac{\delta loss}{\delta a}\cdot \frac{\delta a}{\delta b}; \frac{\delta a}{\delta b} = \frac{\delta XW + b \cdot I_{n x 1}}{\delta b} = I_{n x 1} \Rightarrow \frac{\delta loss}{\delta b} = \frac{2}{n} (a - y)^T \cdot I_{n x 1}$$


In [51]:
dloss_da = 2 / 3 * (a - y_tensor)
dloss_dW = 2 / 3 * (a - y_tensor) @ X_tensor
dloss_db = 2 / 3 * (a - y_tensor) @ torch.ones(len(a))

dloss_da, dloss_dW, dloss_db

(tensor([0.8407, 0.9466, 0.9264], grad_fn=<MulBackward0>),
 tensor([1.8930], grad_fn=<SqueezeBackward4>),
 tensor(2.7138, grad_fn=<DotBackward0>))

In [52]:
loss = errors.mean()
loss.backward()

In [53]:
W.grad, b.grad

(tensor([1.8930]), tensor([2.7138]))

Let's make a gradient step and inspect the loss change!

In [54]:
nu = 0.01 # learning rate
W = W - W.grad * nu
b = b - b.grad * nu

In [55]:
print(f"Initial loss: {loss.item():.4f}")
a = X_tensor @ W + b
errors = (a - y_tensor) ** 2
loss = errors.mean().item()
print(f"New loss: {loss:.4f}")

Initial loss: 1.8459
New loss: 1.7380


We can see a solid loss decrease even after 1 GD step!

### 3. My first neural network

In [56]:
from copy import deepcopy

import numpy as np
import pandas as pd
import plotly.express as px
import torch
from IPython.display import clear_output, display
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader, Dataset
from tqdm.notebook import tqdm

In [57]:
# !wget  -O 'data/sem_1_housing_data.csv' -q 'https://www.dropbox.com/s/6dxq90t0prn2vaw/_train_sem2.csv?dl=0'

In [58]:
df = pd.read_csv('data/sem_1_housing_data.csv')
print(df.shape)
df.head()

(1460, 81)


,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [59]:
data = df.select_dtypes(['int64', 'float64'])
data

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
0,1,60,65.0,8450,7,5,2003,2003,196.0,706,...,0,61,0,0,0,0,0,2,2008,208500
1,2,20,80.0,9600,6,8,1976,1976,0.0,978,...,298,0,0,0,0,0,0,5,2007,181500
2,3,60,68.0,11250,7,5,2001,2002,162.0,486,...,0,42,0,0,0,0,0,9,2008,223500
3,4,70,60.0,9550,7,5,1915,1970,0.0,216,...,0,35,272,0,0,0,0,2,2006,140000
4,5,60,84.0,14260,8,5,2000,2000,350.0,655,...,192,84,0,0,0,0,0,12,2008,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,60,62.0,7917,6,5,1999,2000,0.0,0,...,0,40,0,0,0,0,0,8,2007,175000
1456,1457,20,85.0,13175,6,6,1978,1988,119.0,790,...,349,0,0,0,0,0,0,2,2010,210000
1457,1458,70,66.0,9042,7,9,1941,2006,0.0,275,...,0,60,0,0,0,0,2500,5,2010,266500
1458,1459,20,68.0,9717,5,6,1950,1996,0.0,49,...,366,0,112,0,0,0,0,4,2010,142125


In [60]:
X = data.drop(columns=['Id', 'SalePrice']).fillna(data.mean()).values.astype(np.float32)
y = data.SalePrice.values.astype(np.float32)

In [61]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [62]:
ss = StandardScaler() # Standardize features by removing the mean and scaling to unit variance. https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html
X_train = ss.fit_transform(X_train)
X_test = ss.transform(X_test)

In [63]:
ss2 = StandardScaler() 
y_train = ss2.fit_transform(y_train[:, None]).reshape(-1)
y_test = ss2.transform(y_test[:, None]).reshape(-1)

In [64]:
# Constants
SEED = 42 # random seed for reproducibility
LR = 3e-2 # learning rate, controls the speed of the training
WEIGHT_DECAY = 1e-3 # lambda for L2 reg. ()
NUM_EPOCHS = 10 # num training epochs (how many times each instance will be processed)
GAMMA = 0.9995 # learning rate scheduler parameter
BATCH_SIZE = 64 # training batch size
EVAL_BATCH_SIZE = 300 # evaluation batch size.
DEVICE = 'cpu' #'cuda' # device to make the calculations on
PATIENCE = 5 #  for Early stopping

In [65]:
# Train - validate split
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size = 0.2, random_state = SEED)

In [66]:
# Initialize the DataObject, which must return an element (features vector x and target value y)
# for a given idx. This class must also have a length atribute
class MyDataset(Dataset):
    def __init__(self, X, y):
        super().__init__() # to initialize the parent class
        self.X = X
        self.y = y
        self.len = len(X)

    def __len__(self): # We use __func__ for implementing in-built python functions
        return self.len

    def __getitem__(self, index):
        return self.X[index], self.y[index]

In [67]:
# Initialize DataLoaders - objects, which sample instances from DataObject-s
train_dl = DataLoader(
    MyDataset(X_train, y_train),
    batch_size = BATCH_SIZE,
    shuffle = True
)

val_dl = DataLoader(
    MyDataset(X_val, y_val),
    batch_size = EVAL_BATCH_SIZE,
    shuffle = False
)

test_dl = DataLoader(
    MyDataset(X_test, y_test),
    batch_size = EVAL_BATCH_SIZE,
    shuffle = False
)

data_loaders = {'train': train_dl, 'val': val_dl, 'test': test_dl}

In [68]:
class Model(nn.Module):
    def __init__(self, in_features = 36, out_features = 1):
        super().__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.hidden_size = int(np.sqrt(in_features + out_features))

        self.sequential = nn.Sequential( # NN architecure, where the modules modify the data sequentially
            nn.Linear(in_features, self.hidden_size), # Linear transformation
            nn.ReLU(), # Activation function 
            nn.Linear(self.hidden_size, self.out_features) # Another Linear transformation
        )

    def forward(self, x): # In the forward function, you define how your model runs, from input to output 
        x = self.sequential(x)
        return x

In [69]:
torch.manual_seed(SEED) # Fix random seed to have reproducible weights of model layers

model = Model()
model.to(DEVICE)

loss_fn = nn.MSELoss() # Loss function, which our model will try to minimize
# Initialize GD method, which will update the weights of the model
optimizer = torch.optim.AdamW(model.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)
# Initialize learning rate scheduler, which will decrease LR according to some rule
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=GAMMA)

In [70]:
# Training loop
metrics_dict = {
    "Epoch": [],
    "Train RMSE": [],
    "Val RMSE": [],
}

# Train loop
for epoch in tqdm(range(100)):
    metrics_dict["Epoch"].append(epoch)
    for stage in ['train', 'val']:
        # Whether to start building a graph for a backward pass
        with torch.set_grad_enabled(stage == 'train'): 
            if stage == 'train':
                model.train() # Enable some "special" layers (will speak about later)
            else:
                model.eval() # Disable some "special" layers (will speak about later)

            loss_at_stage = 0 
            for batch in data_loaders[stage]:
                x_batch, y_batch = batch
                x_batch, y_batch = x_batch.to(DEVICE), y_batch.to(DEVICE)

                y_pred = model(x_batch).view(-1) # forward pass: model(x_batch) -> calls forward()
                loss = loss_fn(y_pred, y_batch) # ¡Important! y_pred is always the first arg
                if stage == "train":
                    loss.backward() # Calculate the gradients of all the parameters wrt loss
                    optimizer.step() # Update the parameters
#                     scheduler.step()
                    optimizer.zero_grad() # Zero the saved gradient
                with torch.no_grad():
                    loss_at_stage += (torch.square((y_pred - y_batch)).sum()).item()
            rmse_at_stage = (loss_at_stage / len(data_loaders[stage].dataset)) ** (1/2)
            metrics_dict[f"{stage.title()} RMSE"].append(rmse_at_stage)
            
    clear_output(wait=True)
    display(pd.DataFrame(metrics_dict))
        
    # Early stopping
    val_rmse = metrics_dict["Val RMSE"]
    # If less than PATIENCE observations, cannot activate ES
    if stage == "val" and len(val_rmse) > PATIENCE:
        i_last_valid_obs = -min(len(val_rmse), PATIENCE + 1)
        if rmse_at_stage < min(val_rmse[i_last_valid_obs:-1]):
            best_model_state_dict = model.state_dict()
        elif min(val_rmse[i_last_valid_obs:-1]) == val_rmse[i_last_valid_obs]:
            print(f"Early stopping, going back to epoch {epoch - PATIENCE}. Best quality:\
            {val_rmse[i_last_valid_obs]:.4f}")
            model.load_state_dict(best_model_state_dict)
            break
    elif stage == "val":
        if min(val_rmse) == val_rmse[-1]:
            best_model_state_dict = deepcopy(model.state_dict())
    
px.line(x=torch.arange(len(metrics_dict["Val RMSE"])), y=metrics_dict["Val RMSE"])

,Epoch,Train RMSE,Val RMSE
0,0,0.684218,0.536091
1,1,0.486245,0.408098
2,2,0.441661,0.394270
3,3,0.429719,0.380614
4,4,0.421298,0.361900
5,5,0.414207,0.389208
6,6,0.415186,0.370607
7,7,0.417935,0.475151
8,8,0.420029,0.400714
9,9,0.387926,0.392100


Early stopping, going back to epoch 4. Best quality:            0.3619


In [71]:
def rmse(preds, y_true):
    return ((preds - y_true) ** 2).mean() ** (1/2)

def eval_loop(model, X_val, X_test, y_val, y_test, model_is_nn=True, reschedule=True, model_name="NN"):
    print(f"Model {model_name}")
    for name, x_eval, y_eval in zip(['val', 'test'], [X_val, X_test], [y_val, y_test]):
        if model_is_nn:
            with torch.no_grad():
                preds = model(torch.Tensor(x_eval)).view(-1)
        else:
            model.fit(X_train, y_train)
            preds = model.predict(x_eval)
        # Don't forget to rescale the results back to measure loss in original space
        if reschedule:
            preds = preds * ss2.scale_ + ss2.mean_
            y_eval = y_eval * ss2.scale_ + ss2.mean_
        rmse_val = rmse(preds, y_eval)
        print(f'{name}: RMSE = {rmse_val:.2f}')

In [72]:
eval_loop(model, X_val, X_test, y_val, y_test, reschedule=True, model_is_nn=True, model_name="NN")

Model NN
val: RMSE = 27949.74
test: RMSE = 31735.22


In [73]:
from sklearn.linear_model import LinearRegression, RidgeCV, LassoCV
eval_loop(LinearRegression(), X_val, X_test, y_val, y_test, model_is_nn=False, model_name="LinReg")
eval_loop(RidgeCV(alphas=[1e-3, 1e-2, 1e-1, 1, 1e+1, 1e+2]),
          X_val, X_test, y_val, y_test, model_is_nn=False, model_name="Ridge")

eval_loop(LassoCV(alphas=[1e-3, 1e-2, 1e-1, 1, 1e+1, 1e+2], precompute=False),
          X_val, X_test, y_val, y_test, model_is_nn=False, model_name="Lasso")

# Little reminder, Ridge Regularization adds a penalty proportional to the sum of the squared coefficients, 
# shrinking coefficients but not setting them to zero. 
# Useful for handling multicollinearity and retaining all features.
# While Lasso Regularization adds a penalty proportional to the sum of the absolute values of 
# the coefficients, potentially setting some coefficients to zero. Useful for feature selection 
# and improving model interpretability.

Model LinReg
val: RMSE = 32098.22
test: RMSE = 38142.45
Model Ridge
val: RMSE = 32390.24
test: RMSE = 38265.15
Model Lasso
val: RMSE = 31992.14
test: RMSE = 38403.42
